In [1]:
!pip install scikeras
!pip install tensorflow
!pip install tensorflow keras scikit-learn
!python -m nltk.downloader punkt
!pip install gensim numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 120.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 26.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.2
    Not uninstalling scikit-learn at /shared-libs/python3.10/py/lib/python3.10/site-packages, outside environment /root/venv
    Can't uninstall 'scikit-learn'. No files were found to uninstall.
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Not uninstalling keras at /shared-libs/python3.10/py/lib/python3.10/site-packages, outside environment /root/venv
    Can't uninstall 'keras'. No files w

In [2]:
import pandas as pd
df = pd.read_csv('/work/merged_data.csv')
merged_df = df
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   UniqueID                 45000 non-null  object
 1   Role                     45000 non-null  object
 2   Current Role Experience  45000 non-null  int64 
 3   Educational Background   45000 non-null  object
 4   Comp/Non                 45000 non-null  object
 5   Level                    45000 non-null  object
 6   Course                   45000 non-null  object
 7   Job Role                 45000 non-null  object
 8   Completion               45000 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.1+ MB


In [3]:
merged_df[merged_df.duplicated()].shape
merged_df = merged_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [4]:
# Create a dictionary of user courses
user_courses = {}
for _, row in merged_df.iterrows():
    email = row['UniqueID']
    course_info = {'course_name': row['Course'], 'completion_status': row['Completion']}
    if email in user_courses:
        user_courses[email].append(course_info)
    else:
        user_courses[email] = [course_info]


In [5]:
job_role = merged_df['Job Role'].unique().tolist()
print(job_role)

['Infrastructure Engineer', 'Operation Analyst', 'Data Analyst', 'Data Scientist']


In [6]:
new_merged_df = merged_df.copy()

In [7]:
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.regularizers import l2, l1_l2
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta
tf.experimental.numpy.experimental_enable_numpy_behavior()


# Splitting the data into train and test sets
job_role_groups = merged_df.groupby('Job Role')['UniqueID'].apply(list).to_dict()

def get_recommendations_hybrid(user_id, df_cleaned):
    # Collaborative Filtering
    train_df_cleaned = df_cleaned[['UniqueID', 'Course', 'Completion', 'Job Role']]
    course_groups = train_df_cleaned.groupby('Course')['UniqueID'].apply(list).to_dict()

    user_courses_cf = train_df_cleaned.loc[train_df_cleaned['UniqueID'] == user_id]
    completed_courses_cf = user_courses_cf[user_courses_cf['Completion'] == 'Yes']['Course'].tolist()

    similar_users_cf = []
    for course in completed_courses_cf:
        similar_users_cf.extend(course_groups.get(course, []))
    similar_users_cf = set(similar_users_cf) - set([user_id])
    recommendations_cf = []
    for similar_user in similar_users_cf:
        similar_user_courses = train_df_cleaned.loc[train_df_cleaned['UniqueID'] == similar_user]
        for _, row in similar_user_courses.iterrows():
            if row['Course'] not in completed_courses_cf:
                recommendations_cf.append(row['Course'])

    # Content-based Filtering
    job_role_groups = merged_df.groupby('Job Role')['UniqueID'].apply(list).to_dict()

    if user_id in df_cleaned['UniqueID'].values:
        user_job_role = df_cleaned.loc[df_cleaned['UniqueID'] == user_id, 'Job Role'].iloc[0]
        similar_users_content = job_role_groups[user_job_role]
    else:
        user_job_role = None
        similar_users_content = job_role_groups
    recommendations_content = []
    for similar_user in similar_users_content:
        if similar_user != user_id:
            similar_user_courses = df_cleaned.loc[df_cleaned['UniqueID'] == similar_user]
            completed_courses_content = similar_user_courses[similar_user_courses['Completion'] == 'Yes']['Course'].tolist()
            for _, row in similar_user_courses.iterrows():
              if row['Course'] not in completed_courses_cf and row['Course'] not in completed_courses_content:
                recommendations_content.append(row['Course'])

    # Combine recommendations from both methods
    hybrid_recommendations = recommendations_cf + recommendations_content
    return hybrid_recommendations

2024-05-25 13:00:59.066224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-25 13:00:59.181529: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-25 13:00:59.181555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-25 13:00:59.199726: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-25 13:00:59.851011: W tensorflow/stream_executor/platform/de

In [8]:
le = LabelEncoder()
merged_df['CourseID'] = le.fit_transform(merged_df['Course'])
for col in merged_df.select_dtypes(include=['object', 'int64']):
  if col != 'Course':
    merged_df[col] = le.fit_transform(merged_df[col])

# Create a dictionary to map encoded IDs to original course names
course_names = dict(zip(le.transform(merged_df['CourseID']), merged_df['Course']))

train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# ---- Feature & Target
# Step 4: Convert categorical variables to numerical for neural network

course_groups = train_df.groupby('CourseID')['UniqueID'].apply(list).to_dict()

X_train = train_df[['UniqueID', 'CourseID', 'Job Role', 'Level']]
y_train = train_df['Completion']
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

X_test = test_df[['UniqueID', 'CourseID', 'Job Role', 'Level']]
y_test = test_df['Completion']
encoder = LabelEncoder()
y_test = encoder.fit_transform(y_test)

# Clipping the input
X_train_clipped = tf.clip_by_value(X_train, 0, len(encoder.classes_) - 1)
X_test_clipped = tf.clip_by_value(X_test, 0, len(encoder.classes_) - 1)

2024-05-25 13:01:01.191982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-25 13:01:01.192852: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-25 13:01:01.192881: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-eb85ba0c-8f04-478c-a7e1-467751486180): /proc/driver/nvidia/version does not exist
2024-05-25 13:01:01.197858: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Step 5: Define and train
# ---- LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

lstm_model = Sequential([
    Embedding(input_dim=len(encoder.classes_), output_dim=64, input_length=4),
    Bidirectional(LSTM(128, return_sequences=True)),
    #BatchNormalization(),
    #Dropout(0.2),
    Bidirectional(LSTM(64)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, activation='relu', kernel_regularizer=l2(0.1)),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


lstm_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#lstm_model.fit(X_train_clipped, y_train, epochs=5, batch_size=32, validation_data=(X_test_clipped, y_test))


def get_recommendations_lstm(user_id, user_job_role):
    user_data = train_df[train_df['UniqueID'] == user_id]

    if not user_data.empty and not user_data['Job Role'].empty:
        user_course_data = user_data[['UniqueID', 'Job Role', 'CourseID', 'Level']]

        user_course_data = user_course_data.values.reshape(-1, 4)
        user_course_data = tf.clip_by_value(user_course_data, 0, len(encoder.classes_)-1)
        completion_prob = lstm_model.predict(user_course_data)[0]

        if completion_prob > 0.5:
            completed_courses_lstm = user_course_data[tf.cast(user_course_data[:, -1], tf.float32) > 0.5][:, -1].tolist()
        else:
            completed_courses_lstm = []

        recommendations_lstm = []
        for course in completed_courses_lstm:
            similar_courses_for_job_role = job_role_groups.get(user_job_role, [])
            for similar_course in similar_courses_for_job_role:
                if similar_course not in completed_courses_lstm:
                    recommendations_lstm.append(similar_course)

        return recommendations_lstm
    else:
        return []

# Hybrid Recommendation incorporating lstm
def get_recommendations_deep_hybrid_lstm(user_id, user_job_role):
    try:
        recommendations_cf = get_recommendations_hybrid(user_id, train_df)
        recommendations_lstm = get_recommendations_lstm(user_id, user_job_role)

        hybrid_recommendations = recommendations_cf + recommendations_lstm
        unique_recommendations = list(set(hybrid_recommendations))

        return unique_recommendations
    except IndexError as e:
        print("Error:", e)
        return []

In [10]:
courselink_df = pd.read_csv('/work/courselink.csv')

In [11]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=f1dc03a7560fd2bfa57e784a43165029e8fac6660dd34a583082478d4201c81c
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
import anvil.server
import plotly.express as px
anvil.server.connect("server_Q4TBD6K4O4JEZYVUF6WQQ7RG-KPFNPWJIXDJBN5UF")

@anvil.server.callable
def get_recommendations_deep_hybrid_lstm(user_id, user_job_role):
    try:
        recommendations_cf = get_recommendations_hybrid(user_id, merged_df)
        recommendations_lstm = get_recommendations_lstm(user_id, user_job_role)

        # Combine recommendations from all methods
        hybrid_recommendations = recommendations_cf + recommendations_lstm
        unique_recommendations = list(set(hybrid_recommendations))

        # Get the links for the recommended courses
        recommendations_with_links = []
        for course in unique_recommendations:
            link = courselink_df[courselink_df['Course'] == course]['Link'].values[0]
            recommendations_with_links.append({'course': course, 'link': link})
        
        return recommendations_with_links
    except IndexError as e:
        print("Error:", e)
        return []


@anvil.server.callable
def get_user_courses(email):
    courses = user_courses.get(email, [])
    
    user_data = new_merged_df[new_merged_df['UniqueID'] == email]
    job_role = user_data['Job Role'].iloc[0]
    
    return {'courses': courses, 'job_role': job_role}


@anvil.server.callable
def get_job_roles():
    return job_role

@anvil.server.callable
def get_completion_by_job_role():
    # Aggregate data
    completion_counts = new_merged_df.groupby(['Job Role', 'Completion']).size().reset_index(name='Count')

    fig = px.bar(completion_counts, x='Job Role', y='Count', color='Completion',
                 title='Course Completion by Job Role', barmode='group',
                 color_discrete_map={'No': 'indianred', 'Yes': 'lightgreen'})

    return fig

@anvil.server.callable
def get_user_completion(email):
    # Filter for user
    user_data = new_merged_df[new_merged_df['UniqueID'] == email]

    completion_counts = user_data.groupby('Completion').size().reset_index(name='Count')

    fig = px.pie(completion_counts, names='Completion', values='Count',
                 title=f'Course Completion for {email}')
    fig.update_traces(marker=dict(colors=['indianred', 'lightgreen']))

    return fig

@anvil.server.callable
def get_user_completion_by_job_role(email):
    # Filter data for the given user
    user_data = new_merged_df[new_merged_df['UniqueID'] == email]
    user_job_role = user_data['Job Role'].iloc[0]  # Get the job role of the logged-in user
    
    # Filter data based on the user's job role
    user_job_role_data = df[df['Job Role'] == user_job_role]
    
    # Aggregate data by courses and completion status
    completion_counts = user_job_role_data.groupby(['Course', 'Completion']).size().reset_index(name='Count')
    
    # Create the plot
    fig = px.bar(completion_counts, x='Course', y='Count', color='Completion',
                 title=f'Course Completion for {user_job_role} by Course', barmode='group',
                 color_discrete_map={'No': 'indianred', 'Yes': 'lightgreen'})
    
    return fig
 

#905c024e

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eb85ba0c-8f04-478c-a7e1-467751486180' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>